In [1]:
##Import libraries 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime, timedelta

# import all libraries and dependencies for data visualization
pd.options.display.float_format='{:.4f}'.format
plt.rcParams['figure.figsize'] = [8,8]
pd.set_option('display.max_columns', 350)
pd.set_option('display.max_colwidth', -1) 
pd.set_option("display.max_rows", 50)
sns.set(style='darkgrid')
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker


# import all libraries and dependencies for machine learning
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.base import TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler

from sklearn.compose import make_column_transformer


import statsmodels.api as sm
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

from scipy import stats
from scipy.stats import norm, kurtosis, skew

# Import specific libraries
import statsmodels.api as sm
from statsmodels.stats import diagnostic as diag
from statsmodels.stats.outliers_influence import variance_inflation_factor

pd.set_option('expand_frame_repr', False)


# Models
import mlxtend
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.svm import SVR
from mlxtend.regressor import StackingCVRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

# Stats
from scipy.stats import skew, norm
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

# Misc
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA

pd.set_option('display.max_columns', None)

# Ignore useless warnings
import warnings
warnings.filterwarnings(action="ignore")
pd.options.display.max_seq_items = 8000
pd.options.display.max_rows = 8000


In [2]:
%store -r all_data
%store -r y_train
%store -r data_4
%store -r data_5
%store -r data_1
%store -r data_2
%store -r data_3
%store -r data_4
%store -r data_5
%store -r all_2
%store -r data_0
%store -r mae_data
%store -r data_mae
%store -r data_mae_11
%store -r y_train_2
%store -r train_mae
%store -r test_mae
%store -r cols




In [3]:
mae_data = train_mae
mae_data.drop("index", axis = 1, inplace = True)
mae_data.head()

,Street,LandSlope,OverallCond,YearBuilt,YearRemodAdd,BsmtQual,BsmtExposure,BsmtFinType1,HeatingQC,BsmtHalfBath,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageFinish,OpenPorchSF,TotalExtraPoints,ConstructArea,TotBathrooms,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_120,MSSubClass_160,MSSubClass_180,MSSubClass_190,Alley_NA,LotConfig_Inside,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_RRAn,Condition2_Artery,HouseStyle_15Fin,HouseStyle_15Unf,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gambrel,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_CompShg,RoofMatl_TarGrv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_HdBoard,Exterior1st_Stucco,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_BrkFace,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_WdSdng,MasVnrType_BrkFace,MasVnrType_Stone,Foundation_Slab,Electrical_FuseF,GarageType_CarPort,GarageType_Detchd,GarageType_NA,MiscFeature_Othr,MiscFeature_Pool,MoSold_1,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,MoSold_11,MoSold_12,SaleCondition_AdjLand,SaleCondition_Alloca,Remod,SalePrice
0,0.0000,0.0000,0.0000,-0.6522,-0.2368,0.0000,0.0000,0.4000,0.0000,0.0000,0.0000,1.0000,0.8715,0.0000,-1.0000,0.0000,0.2212,0.0000,0.1474,3.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,12.2477
1,0.0000,0.0000,3.0000,-0.0652,0.4737,0.0000,1.9890,0.2000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.7525,0.4000,0.0823,1.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,12.1090
2,0.0000,0.0000,0.0000,-0.6087,-0.2105,0.0000,1.0000,0.4000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.1316,0.6000,0.2520,3.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000

In [ ]:
# len_test = mae_data.shape[0] - mae_data.iloc[:len(y_train), :].shape[0]
# y_test = np.zeros((len_test,1))
# y_test = pd.DataFrame(y_test)
# mae_data["SalePrice"] = pd.concat((y_train, y_test), axis = 0)

# mae_data.head(2)

In [4]:
# # X = mae_data.iloc[:len(y_train), :].drop("SalePrice", axis = 1)
# X = mae_data[mae_data.SalePrice > 0].drop("SalePrice", axis = 1)
train_labels = y_train

# # #Rescaling X
# #scale = RobustScaler()
# #X = scale.fit_transform(X)


In [5]:
X = mae_data.drop("SalePrice", axis =1)

In [6]:
X.shape

(1449, 108)

In [7]:
#Setup cross validation and define error metrics

In [8]:
# Setup cross validation folds
kf = KFold(n_splits=12, random_state=42, shuffle=True)

In [9]:
#SETUP MODELS

In [10]:
# Define error metrics
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def cv_rmse(model, X = X):
    rmse = np.sqrt(-cross_val_score(model, X, train_labels, scoring="neg_mean_squared_error", cv=kf))
    return (rmse)

In [11]:
# Light Gradient Boosting Regressor
lightgbm = LGBMRegressor(objective='regression', 
                       num_leaves=6,
                       learning_rate=0.01, 
                       n_estimators=7000,
                       max_bin=200, 
                       bagging_fraction=0.8,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.2,
                       feature_fraction_seed=8,
                       min_sum_hessian_in_leaf = 11,
                       verbose=-1,
                       random_state=42)

# XGBoost Regressor
xgboost = XGBRegressor(learning_rate=0.01,
                       n_estimators=6000,
                       max_depth=4,
                       min_child_weight=0,
                       gamma=0.6,
                       subsample=0.7,
                       colsample_bytree=0.7,
                       objective ='reg:squarederror',
                       nthread=-1,
                       scale_pos_weight=1,
                       seed=27,
                       reg_alpha=0.00006,
                       random_state=42)

# Ridge Regressor
ridge_alphas = [1e-15, 1e-10, 1e-8, 9e-4, 7e-4, 5e-4, 3e-4, 1e-4, 1e-3, 5e-2, 1e-2, 0.1, 0.3, 1, 3, 5, 10, 15, 18, 20, 30, 50, 75, 100]
ridge = make_pipeline(RobustScaler(), RidgeCV(alphas=ridge_alphas, cv=kf))

# Support Vector Regressor
svr = make_pipeline(RobustScaler(), SVR(C= 20, epsilon= 0.008, gamma=0.0003))

# Gradient Boosting Regressor
gbr = GradientBoostingRegressor(n_estimators=6000,
                                learning_rate=0.01,
                                max_depth=4,
                                max_features='sqrt',
                                min_samples_leaf=15,
                                min_samples_split=10,
                                loss='huber',
                                random_state=42)  

# Random Forest Regressor
rf = RandomForestRegressor(n_estimators=1200,
                          max_depth=15,
                          min_samples_split=5,
                          min_samples_leaf=5,
                          max_features=None,
                          oob_score=True,
                          random_state=42)

# Stack up all the models above, optimized using xgboost
stack_gen = StackingCVRegressor(regressors=(xgboost, lightgbm, svr, ridge, gbr, rf),
                                meta_regressor=xgboost,
                                use_features_in_secondary=True)

In [12]:
#TRAIN INDIVIDUAL MODELS

In [13]:
#Get cross validation scores for each model

In [14]:
scores = {}
score = cv_rmse(lightgbm)
print("lightgbm: {:.4f} ({:.4f})".format(score.mean(), score.std()))
scores['lgb'] = (score.mean(), score.std())

lightgbm: 0.1251 (0.0170)


In [15]:
score = cv_rmse(xgboost)
print("xgboost: {:.4f} ({:.4f})".format(score.mean(), score.std()))
scores['xgb'] = (score.mean(), score.std())


xgboost: 0.1432 (0.0182)


In [16]:
score = cv_rmse(svr)
print("SVR: {:.4f} ({:.4f})".format(score.mean(), score.std()))
scores['svr'] = (score.mean(), score.std())

SVR: 0.1259 (0.0186)


In [17]:
score = cv_rmse(ridge)
print("ridge: {:.4f} ({:.4f})".format(score.mean(), score.std()))
scores['ridge'] = (score.mean(), score.std())

ridge: 0.1275 (0.0160)


In [18]:
score = cv_rmse(rf)
print("rf: {:.4f} ({:.4f})".format(score.mean(), score.std()))
scores['rf'] = (score.mean(), score.std())

rf: 0.1419 (0.0171)


In [19]:
score = cv_rmse(gbr)
print("gbr: {:.4f} ({:.4f})".format(score.mean(), score.std()))
scores['gbr'] = (score.mean(), score.std())

KeyboardInterrupt: 

In [ ]:
scores

In [ ]:
#FIT THE MODELS

In [ ]:
print('stack_gen')
stack_gen_model = stack_gen.fit(np.array(X), np.array(train_labels))

In [ ]:
print('lightgbm')
lgb_model_full_data = lightgbm.fit(X, train_labels)

In [ ]:
print('xgboost')
xgb_model_full_data = xgboost.fit(X, train_labels)

In [ ]:
print('Svr')
svr_model_full_data = svr.fit(X, train_labels)

In [ ]:
print('Ridge')
ridge_model_full_data = ridge.fit(X, train_labels)

In [ ]:
print('RandomForest')
rf_model_full_data = rf.fit(X, train_labels)

In [ ]:
print('GradientBoosting')
gbr_model_full_data = gbr.fit(X, train_labels)

In [ ]:
#BLEND MODELS AND MAKE PREDICTIONS
print(ridge_model_full_data.predict(X), svr_model_full_data.predict(X), gbr_model_full_data.predict(X),xgb_model_full_data.predict(X),lgb_model_full_data.predict(X), rf_model_full_data.predict(X),stack_gen_model.predict(np.array(X)))

In [ ]:
# # Blend models in order to make the final predictions more robust to overfitting: BEST SCORE SO FAR COMBINATION WITH DATA_1
# ##RMSE: 0.0700069/sCORE: 0.11893 PLACE 725
# def blended_predictions(X):
#     return ((0.25 * ridge_model_full_data.predict(X)) + \
#             (0.0 * svr_model_full_data.predict(X)) + \
#             (0.25 * gbr_model_full_data.predict(X)) + \
#             (0.0 * xgb_model_full_data.predict(X)) + \
#             (0.15 * lgb_model_full_data.predict(X)) + \
#             (0.0 * rf_model_full_data.predict(X)) + \
#             (0.35 * stack_gen_model.predict(np.array(X))))

In [ ]:
# Blend models in order to make the final predictions more robust to overfitting: TEST ALL_2

def blended_predictions(X):
    return ((0.0 * ridge_model_full_data.predict(X)) + \
            (0.2 * svr_model_full_data.predict(X)) + \
            (0.05 * gbr_model_full_data.predict(X)) + \
            (0.2 * xgb_model_full_data.predict(X)) + \
            (0.05 * lgb_model_full_data.predict(X)) + \
            (0.1 * rf_model_full_data.predict(X)) + \
            (0.4 * stack_gen_model.predict(np.array(X))))

In [ ]:
### BEST VALIDATION SCORE WITH data_3, but not better in leaderboard than data1 
#RMSE= 0.066374 /SCORE= 0.11907

# def blended_predictions(X):
#     return ((0.15 * ridge_model_full_data.predict(X)) + \
#             (0.0 * svr_model_full_data.predict(X)) + \
#             (0.3 * gbr_model_full_data.predict(X)) + \
#             (0.0 * xgb_model_full_data.predict(X)) + \
#             (0.2 * lgb_model_full_data.predict(X)) + \
#             (0.0 * rf_model_full_data.predict(X)) + \
#             (0.35 * stack_gen_model.predict(np.array(X))))

In [ ]:
# Blend models in order to make the final predictions more robust to overfitting
# def blended_predictions(X):ponderacion vieja
#     return ((0.1 * ridge_model_full_data.predict(X)) + \
#             (0.2 * svr_model_full_data.predict(X)) + \
#             (0.1 * gbr_model_full_data.predict(X)) + \
#             (0.1 * xgb_model_full_data.predict(X)) + \
#             (0.1 * lgb_model_full_data.predict(X)) + \
#             (0.05 * rf_model_full_data.predict(X)) + \
#             (0.35 * stack_gen_model.predict(np.array(X))))

In [ ]:
blended_predictions(X)

In [ ]:
# Get final precitions from the blended model
blended_score = rmsle(train_labels, blended_predictions(X))
scores['blended'] = (blended_score, 0)
print('RMSLE score on train data:')
print(blended_score)

In [ ]:
#IDENTIFY THE BEST PERFORMING MODEL

In [ ]:
# Plot the predictions for each model
sns.set_style("white")
fig = plt.figure(figsize=(24, 12))

ax = sns.pointplot(x=list(scores.keys()), y=[score for score, _ in scores.values()], markers=['o'], linestyles=['-'])
for i, score in enumerate(scores.values()):
    ax.text(i, score[0] + 0.002, '{:.6f}'.format(score[0]), horizontalalignment='left', size='large', color='black', weight='semibold')

plt.ylabel('Score (RMSE)', size=20, labelpad=12.5)
plt.xlabel('Model', size=20, labelpad=12.5)
plt.tick_params(axis='x', labelsize=13.5)
plt.tick_params(axis='y', labelsize=12.5)

plt.title('Scores of Models', size=20)

plt.show()

In [ ]:
test_mae.shape, train_mae.shape

In [ ]:
train_mae.head()

In [ ]:
##PREDICTION FOR THE TEST SET: We make the prediction using the best working model (blended)
# test = all_2[all_2.SalePrice == 0]

# test = test.drop("SalePrice", axis = 1, inplace = False)
# test.shape
#Robustscaler
# test = RobustScaler().fit_transform(test_mae)
# test = pd.DataFrame(test, columns = cols)
%store -r test_ID

In [ ]:
test_mae.shape

In [ ]:
#submit_june = np.expm1(blended_predictions(test)) ##
submit_july = np.expm1(ridge_model_full_data.predict(test_mae))

# s_july = pd.DataFrame({"Id": test_ID, "SalePrice": submit_july})

In [ ]:
s_june = s_june.set_index("Id")

In [ ]:
# EXPERIMENTO DE LA VIEJA: Fix outlier predictions
q1 = s_june.quantile(0.0045)[0]
q2 = s_june.quantile(0.99)[0]
s_june["SalePrice"] = s_june["SalePrice"].apply(lambda x: x if x > q1 else x*0.77)
s_june["SalePrice"] = s_june["SalePrice"].apply(lambda x: x if x < q2 else x*1.1)
s_june["SalePrice"]  *= 1.001619

In [ ]:
# s_june.to_csv("s_july_2_2.csv")

In [ ]:
s_june.to_csv("s_july_XX.csv")

In [ ]:
s_june